In [1]:
import requests
import pandas as pd
import time

In [2]:
api_key = "3a2957dbae161d9aac23aadb136880f304a1653c2003ca11cc478e2f30f8"

In [3]:
payload_projects = {'api-key': api_key} 
url_projects = 'https://data-api.defipulse.com/api/v1/defipulse/api/GetProjects'

In [4]:
res_proj = requests.get(url_projects,params= payload_projects).json()


In [5]:
res_proj

[{'category': 'Lending',
  'variability': 'medium',
  'website': 'https://aave.com',
  'chain': 'Multichain,Ethereum-Polygon',
  'id': 22,
  'name': 'Aave',
  'value': {'tvl': {'USD': {'value': 11867702865, 'relative_1d': -12.16},
    'ETH': {'value': 5111027.16862333, 'relative_1d': -1.63},
    'BTC': {'value': 335471.95553274045, 'relative_1d': -2.7800000000000002}},
   'total': {'USD': {'value': 11867702865, 'relative_1d': -12.16},
    'ETH': {'value': 1127073.139198, 'relative_1d': -0.31},
    'BTC': {'value': 61535.69738088999, 'relative_1d': 0.77}},
   'balance': {'ERC20': {'DAI': {'value': 601554531.0961745,
      'relative_1d': 7.42}}}},
  'contributesTo': None,
  'relative_1d': -5.65,
  'timestamp': 1623484800},
 {'category': 'Lending',
  'chain': 'Ethereum',
  'id': 0,
  'name': 'Maker',
  'value': {'tvl': {'USD': {'value': 7647090024, 'relative_1d': -4.39},
    'ETH': {'value': 3293348.7902565915, 'relative_1d': 0.97},
    'BTC': {'value': 216165.18998398355, 'relative_1d': 

In [6]:
#1. project - Project name. Corresponds to the slug of a project when clicking on it on defipulse.com
#2. period - 1w, 1m, 3m, 1y or all . 
#            (The 1w period will return to hourly values, all other periods will return daily values. Default all)
#3. lenght - Desired length of data set in hours or days. 
#           (Can be used as an alternative to period for a more granular control over the returned data set.)
#4. resolution -  Desired resolution hours or history(=days),
#                 (used together with length for a more granular control over the returned data set)
#5. category - Return data for a specific category only. For historical reasons, project=all is required for this to work.
#              (Current categories: lending, dexes, derivatives, payments, assets)
url = f"https://data-api.defipulse.com/api/v1/defipulse/api/GetHistory"
def get_historical_tvl_by_project_name(pname,retry=0):
    pname = pname.replace(" ","-")
    payload = {'api-key': api_key
                ,'project': pname
    #            ,'period':''
                ,'length':'90'
                ,'resolution':'days'
    #            ,'category':'all'
              }
    res = requests.get(url,params= payload)
    if len(res.text)<100 and retry<3:
        retry = retry+1
        print(f'Retrying for {pname} : {retry}')
        time.sleep(3)
        return get_historical_tvl_by_project_name(pname,retry)
    elif retry>3:
        return None
    df_res = pd.read_json(res.text)
    new_cns = []
    for cn in df_res.columns:
        if cn == 'timestamp':
            new_cn = cn
        else:
            new_cn = f'{pname}_{cn}'
        new_cns.append(new_cn)
    df_res.columns = new_cns
    #display(df_res)
    return  df_res

In [7]:
df_parent = None 
for index, row in df_proj.iterrows():
    pname = row['name']
    df_child =  get_historical_tvl_by_project_name(pname)
    if len(df_child)==0:
        continue
    if index == 0:
        df_parent = df_child.copy()
    else:
        df_parent = pd.merge(df_parent,df_child,on='timestamp',how='outer',indicator=False)
        #display(df_parent)
        #break

NameError: name 'df_proj' is not defined

In [8]:
df_parent.sort_values(by=['timestamp'],inplace=True)
df_parent.reset_index(drop=True,inplace=True)
df_parent.to_csv('HistoricalDailyTVL.csv',index=None)
df_parent

,timestamp,Maker_tvlUSD,Maker_tvlETH,Maker_BTC,Maker_ETH,Maker_DAI,Aave_tvlUSD,Aave_tvlETH,Aave_BTC,Aave_ETH,...,Auctus_ETH,UniTrade_tvlUSD,UniTrade_tvlETH,UniTrade_BTC,UniTrade_ETH,UniTrade_DAI,Connext_tvlUSD,Connext_tvlETH,Connext_BTC,Connext_DAI
0,2020-11-20 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-11-21 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-11-22 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-11-23 00:00:00,2.646823e+09,4.725288e+06,0.000000,2.683540e+06,3.288758e+08,1.628763e+09,2.907779e+06,15404.128288,396068.475796,...,122.440908,75235.0,134.314636,0.0,52.021607,NaN,12796.0,22.844289,0.0,12768.952584
4,2020-11-24 00:00:00,2.768471e+09,4.572432e+06,14899.598316,2.670361e+06,3.310137e+08,1.682207e+09,2.778349e+06,15514.524367,393091.604811,...,122.440908,59852.0,98.852131,0.0,33.997634,NaN,12785.0,21.115827,0.0,12768.952584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,2021-02-20 00:00:00,7.139396e+09,3.646698e+06,13701.768639,2.832263e+06,9.042741e+06,5.605657e+09,2.863287e+06,9694.338876,466637.198922,...,188.841066,97192.0,49.644238,0.0,15.466924,NaN,12781.0,6.528346,0.0,12768.952584
94,2021-02-21 00:00:00,7.054215e+09,3.673630e+06,13836.880361,2.853782e+06,8.528125e+06,5.680520e+09,2.958250e+06,9504.549051,472875.825472,...,98.841066,86091.0,44.833692,0.0,13.772336,NaN,12779.0,6.654932,0.0,12768.952584
95,2021-02-22 00:00:00,7.159530e+09,3.695489e+06,13937.296425,2.860040e+06,9.250082e+06,5.660620e+09,2.921807e+06,9491.582277,459794.447375,...,103.435370,92212.0,47.596484,0.0,15.232484,NaN,12777.0,6.595023,0.0,12768.952584
96,2021-02-22 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
